In [1]:
import sys
from pathlib import Path

# Set up paths
VIGNETTE_NAME = "test"

project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

VIGNETTE_PATH = project_root / "test_data" / VIGNETTE_NAME
rgb_path = VIGNETTE_PATH / "rgb2.png" # "rgb.HEIC" # This could be heic
depth_path = VIGNETTE_PATH / "depth.png"

In [ ]:
import time
import numpy as np
import open3d as o3d
from typing import List, Dict, Union, Sequence, Optional

GeometrySpec = Dict[str, object]
GeometryLike = Union[o3d.geometry.Geometry, GeometrySpec]

def _build_geometry(spec: GeometrySpec) -> o3d.geometry.Geometry:
    """
    Build an Open3D geometry from a simple spec dictionary.
    Supported formats:
      - Point cloud:
          {"type": "pointcloud", "points": (N,3), "colors": (N,3) optional in [0,1]}
      - Line set:
          {"type": "lines", "points": (N,3), "lines": (M,2 int), "colors": (M,3) optional}
      - Mesh (triangle):
          {"type": "mesh", "vertices": (N,3), "triangles": (M,3 int),
           "vertex_colors": (N,3) optional, "vertex_normals": (N,3) optional}
      - Plane (as a simple rectangle):
          {"type":"plane", "center":[3], "normal":[3], "size":[w,h],
           "color":[3] optional in [0,1]}
    """
    gtype = spec.get("type", "").lower()

    if gtype == "pointcloud":
        pcd = o3d.geometry.PointCloud()
        pts = np.asarray(spec["points"], dtype=np.float64)
        pcd.points = o3d.utility.Vector3dVector(pts)
        if "colors" in spec:
            cols = np.asarray(spec["colors"], dtype=np.float64)
            pcd.colors = o3d.utility.Vector3dVector(cols)
        return pcd

    elif gtype == "lines":
        ls = o3d.geometry.LineSet()
        pts = np.asarray(spec["points"], dtype=np.float64)
        lines = np.asarray(spec["lines"], dtype=np.int32)
        ls.points = o3d.utility.Vector3dVector(pts)
        ls.lines = o3d.utility.Vector2iVector(lines)
        if "colors" in spec:
            cols = np.asarray(spec["colors"], dtype=np.float64)
            ls.colors = o3d.utility.Vector3dVector(cols)
        return ls

    elif gtype == "mesh":
        mesh = o3d.geometry.TriangleMesh()
        V = np.asarray(spec["vertices"], dtype=np.float64)
        F = np.asarray(spec["triangles"], dtype=np.int32)
        mesh.vertices = o3d.utility.Vector3dVector(V)
        mesh.triangles = o3d.utility.Vector3iVector(F)
        if "vertex_colors" in spec:
            C = np.asarray(spec["vertex_colors"], dtype=np.float64)
            mesh.vertex_colors = o3d.utility.Vector3dVector(C)
        if "vertex_normals" in spec:
            N = np.asarray(spec["vertex_normals"], dtype=np.float64)
            mesh.vertex_normals = o3d.utility.Vector3dVector(N)
        if not mesh.has_vertex_normals() and not mesh.has_triangle_normals():
            mesh.compute_vertex_normals()
        return mesh

    elif gtype == "plane":
        center = np.asarray(spec.get("center", [0, 0, 0]), dtype=np.float64)
        normal = np.asarray(spec.get("normal", [0, 0, 1]), dtype=np.float64)
        size = spec.get("size", [1.0, 1.0])
        w, h = float(size[0]), float(size[1])

        # build local axes
        n = normal / (np.linalg.norm(normal) + 1e-12)
        # pick an arbitrary axis not parallel to n
        a = np.array([1, 0, 0], dtype=np.float64)
        if abs(np.dot(a, n)) > 0.9:
            a = np.array([0, 1, 0], dtype=np.float64)
        u = np.cross(n, a); u /= (np.linalg.norm(u) + 1e-12)
        v = np.cross(n, u); v /= (np.linalg.norm(v) + 1e-12)

        # rectangle corners
        p0 = center + (-w/2)*u + (-h/2)*v
        p1 = center + ( w/2)*u + (-h/2)*v
        p2 = center + ( w/2)*u + ( h/2)*v
        p3 = center + (-w/2)*u + ( h/2)*v

        mesh = o3d.geometry.TriangleMesh(
            o3d.utility.Vector3dVector(np.vstack([p0, p1, p2, p3])),
            o3d.utility.Vector3iVector(np.array([[0,1,2],[0,2,3]], dtype=np.int32))
        )
        mesh.compute_triangle_normals()
        mesh.compute_vertex_normals()

        color = spec.get("color", [0.8, 0.8, 0.8])
        mesh.paint_uniform_color([float(color[0]), float(color[1]), float(color[2])])
        return mesh

    else:
        raise ValueError(f"Unknown geometry spec type: {gtype}")


def _as_geometry_list(frame_geoms: Sequence[GeometryLike]) -> List[o3d.geometry.Geometry]:
    """Convert a list of mixed geometry inputs (o3d or spec dict) to a list of o3d geometries."""
    built = []
    for g in frame_geoms:
        if isinstance(g, o3d.geometry.Geometry):
            built.append(g)
        elif isinstance(g, dict):
            built.append(_build_geometry(g))
        else:
            raise TypeError("Geometry must be an Open3D geometry or a spec dict.")
    return built


def _apply_camera_view(vis: o3d.visualization.Visualizer, view: Dict[str, object]):
    """
    Apply a simple camera view:
      view = {"eye":[3], "lookat":[3], "up":[3], "fov_deg": float (optional)}
    """
    ctr = vis.get_view_control()
    eye = np.array(view["eye"], dtype=np.float64)
    look = np.array(view["lookat"], dtype=np.float64)
    up = np.array(view.get("up", [0, 1, 0]), dtype=np.float64)

    front = (look - eye)
    nrm = np.linalg.norm(front) + 1e-12
    front = (front / nrm).tolist()
    ctr.set_lookat(look.tolist())
    ctr.set_front(front)
    ctr.set_up(up.tolist())

    # FOV control (optional)
    # Open3D's Python API exposes field of view via get_field_of_view/set_field_of_view in newer versions.
    try:
        if "fov_deg" in view:
            ctr.set_field_of_view(float(view["fov_deg"]))
    except Exception:
        # Older Open3D versions may not support it—silently ignore.
        pass


def animate_scene(
    geometry_frames: List[Sequence[GeometryLike]],
    camera_views: Optional[List[Dict[str, object]]] = None,
    *,
    fps: int = 30,
    width: int = 1280,
    height: int = 720,
    point_size: float = 2.0,
    bg_color: Sequence[float] = (0.0, 0.0, 0.0),
):
    """
    Render an animation where each frame has its own list of geometries and camera view.

    Args:
        geometry_frames: list of frames; each frame is a list of geometries or spec dicts.
        camera_views: optional list of camera dicts, one per frame:
                      {"eye":[x,y,z], "lookat":[x,y,z], "up":[x,y,z], "fov_deg":float}
                      If None, the camera is set once to look at the first frame bbox.
        fps: frames per second.
        width, height: window size.
        point_size: render point size.
        bg_color: background color (r,g,b) in 0..1.
    """
    assert len(geometry_frames) > 0, "No frames provided."
    n_frames = len(geometry_frames)
    if camera_views is not None:
        assert len(camera_views) == n_frames, "camera_views must match number of frames."

    vis = o3d.visualization.Visualizer()
    vis.create_window("Open3D Animation", width=width, height=height)
    opt = vis.get_render_option()
    opt.point_size = float(point_size)
    opt.background_color = np.array(bg_color, dtype=np.float32)

    # Add first frame
    current_geoms = _as_geometry_list(geometry_frames[0])
    for g in current_geoms:
        vis.add_geometry(g)

    # Initial camera if not provided
    if camera_views is None:
        # Aim camera to fit the first frame
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(
            o3d.utility.Vector3dVector(
                np.vstack([
                    np.asarray(getattr(g, "points"), dtype=np.float64)
                    if isinstance(g, o3d.geometry.PointCloud) else
                    np.asarray(g.vertices, dtype=np.float64)
                    if isinstance(g, o3d.geometry.TriangleMesh) else
                    np.asarray(g.points, dtype=np.float64)
                    if isinstance(g, o3d.geometry.LineSet) else
                    np.zeros((0,3), dtype=np.float64)
                    for g in current_geoms
                ]) if len(current_geoms) > 0 else np.zeros((1,3))
            )
        )
        ctr = vis.get_view_control()
        ctr.set_lookat(bbox.get_center().tolist())
        ctr.set_front([0, 0, -1])
        ctr.set_up([0, 1, 0])
        ctr.set_zoom(0.45)
    else:
        _apply_camera_view(vis, camera_views[0])

    vis.poll_events()
    vis.update_renderer()

    dt = 1.0 / max(1, int(fps))

    # Main loop
    for i in range(n_frames):
        # Swap geometries for this frame
        # Remove previous geoms (skip for i == 0 since already added)
        if i > 0:
            for g in current_geoms:
                vis.remove_geometry(g, reset_bounding_box=False)
            current_geoms = _as_geometry_list(geometry_frames[i])
            for g in current_geoms:
                vis.add_geometry(g)

        # Apply per-frame camera if provided
        if camera_views is not None:
            _apply_camera_view(vis, camera_views[i])

        # Draw
        vis.poll_events()
        vis.update_renderer()
        time.sleep(dt)

    vis.destroy_window()

In [4]:
import time
import numpy as np
import open3d as o3d

def play_pointcloud_sequence(points_frames, fps=30, point_size=3.0):
    vis = o3d.visualization.Visualizer()
    vis.create_window("Open3D Point Cloud Animation", width=960, height=720)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_frames[0])
    pcd.paint_uniform_color([0.8, 0.2, 0.8])
    vis.add_geometry(pcd)
    vis.get_render_option().point_size = point_size

    # move camera backward so we can see the cloud
    ctr = vis.get_view_control()
    ctr.set_front([0, 0, -1])
    ctr.set_up([0, 1, 0])
    ctr.set_lookat([0, 0, 0])
    ctr.set_zoom(0.4)

    dt = 1.0 / fps
    for pts in points_frames:
        pcd.points = o3d.utility.Vector3dVector(pts)
        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(dt)

    vis.destroy_window()

# --- Generate test frames ---
points_frames = []
for k in range(60):
    theta = k * (2 * np.pi / 60)
    # Cloud centered 1 m away so camera can see it
    pts = np.random.randn(5000, 3) * 0.1 + np.array([0, 0, 1])
    R = o3d.geometry.get_rotation_matrix_from_xyz((0, theta, 0))
    pts = pts @ R.T
    points_frames.append(pts)

play_pointcloud_sequence(points_frames, fps=30)